In [ ]:
import pandas as pd
import numpy as np
import pickle

# input should be BGRD__signame.pkl (found in GPS4Drugs/data/dzsig) and DZSIG__signame.csv from GPS
# results in the rges_input folder to be used as inputs for RGES score module in MolSearch (rges.py)

In [ ]:
# replace with your signature name
sig_name = 'Myofibroblasts'

with open(f'/rges_input/BGRD__{sig_name}.pkl', 'rb') as f:
    bgrd = pickle.load(f, encoding='latin1')
bgrd.to_csv(f'/rges_input/BGRD__{sig_name}.csv')

In [ ]:
dfs = pd.read_csv(f'/rges_input/DZSIG__{sig_name}.csv', index_col=0)
dfs = dfs.rename_axis('Symbol')
dfs.to_csv(f'/rges_input/DZSIG__{sig_name}.csv')

df1 = pd.read_csv('/rges_input/go_fingerprints_2k_HEPG2_t0.csv', index_col=0)
df2 = pd.read_csv('/rges_input/go_fingerprints_2k_MCF7_t1.csv', index_col=0)
df3 = pd.read_csv('/rges_input/go_fingerprints_2k_PC3_t1.csv', index_col=0)
df4 = pd.read_csv('/rges_input/go_fingerprints_2k_VCAP_t1.csv', index_col=0)

In [ ]:
genes_common = list(set(list(set(dfs.index) & set(df1.index)) + list(set(dfs.index) & set(df2.index)) + list(set(dfs.index) & set(df3.index)) + list(set(dfs.index) & set(df4.index))))

print(len(genes_common))

if len(df1.columns) == len(df2.columns) == len(df3.columns) == len(df4.columns):
    final_df = pd.concat([df1, df2, df3, df4], axis=0).loc[genes_common].groupby(level=0).max().sort_index()
    final_df.to_csv(f'/rges_input/{sig_name}_gene_feature.csv')

    fdf = pd.concat([df1, df2, df3, df4], axis=1).loc[genes_common].sort_index()

    list_for_pickling = []
    for n in range(4):
        cellline_df = fdf.T[len(df1.columns)*n:len(df1.columns)*(n+1)].T
        na_df = cellline_df[cellline_df[cellline_df.columns[0]].isna()]
        list_for_pickling.append(list(np.where(fdf.index.isin(na_df.index))[0]))

    with open(f'/rges_input/NA_IDX_{sig_name}.pkl','wb') as f:
        pickle.dump(list_for_pickling, f)
else:
    print('Wrong cellline files!!!')

56
